In [ ]:
pip install anvil-uplink

: 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import anvil.server
anvil.server.connect("server_ZVQAXY7YIYLEXCBC7F362CET-WPRVZF4A7VJTK334")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

: 

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
r = pd.read_csv('Ratings.csv')
b = pd.read_csv('Books.csv')

: 

In [ ]:
#Rating
r.head()

: 

In [ ]:
r.shape

: 

In [ ]:
for col in r.columns:
    print(f"Number of {col} is {r[col].nunique()}")

: 

In [ ]:
#Books
b.shape

: 

In [ ]:
b.head(4).T

: 

In [ ]:
#check missing value
dataset = [r, b]
for data in dataset:
    print(data.isnull().values.any())

: 

In [ ]:
#check duplicated rows
for data in dataset:
    print(data.duplicated().values.any())

: 

In [ ]:
#There is no duplicate row in both dataset, but we have missing value in book metadata.

: 

In [ ]:
#make a copy 
books = b.copy()

: 

In [ ]:
percent_missing = books.isnull().sum() * 100 / len(books)
missing_value = (pd.DataFrame({'percent_missing': percent_missing})).sort_values(by=['percent_missing'], ascending=False)
missing_value[missing_value['percent_missing']>0]

: 

In [ ]:
#keep important columns, drop the rest
cols_to_keep = ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
books = books[cols_to_keep]

: 

In [ ]:
#display max column
with pd.option_context('display.max_colwidth', None):
    display(books.head(5))

: 

In [ ]:

print(books.columns)

: 

In [ ]:
print(r.columns)

: 

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(r['Book-Rating'], color = '#0047AB')
plt.title('Distribution of Rating', weight='bold', fontsize =20)
plt.xlabel('Book-Rating', fontsize=18)
plt.ylabel('Density', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

: 

In [ ]:
r.describe()

: 

In [ ]:
# Merge the datasets based on book identifier
combined_data = pd.merge(b, r, on='ISBN')

# Display the combined dataset
print(combined_data.head())

: 

In [ ]:
# Let's check authors with biggest number of rates (total number for all books)
high_rating_author = combined_data.groupby('Book-Author')['Book-Rating'].mean().sort_values(ascending = False)
high_rating_author.head(5)

: 

In [ ]:
combined_data.to_csv('merged.csv')

: 

In [ ]:
combined_data.head()

: 

In [ ]:
fd = pd.read_csv('merged.csv')

: 

In [ ]:
fd.shape

: 

In [ ]:
fd.head(4)

: 

In [ ]:
book_pivot= fd.pivot_table(index ='Book-Title', columns= 'Book-Rating', values ='User-ID')

: 

In [ ]:
book_pivot.shape

: 

In [ ]:
from scipy.sparse import csr_matrix

: 

In [ ]:
book_sparse = csr_matrix(book_pivot)

: 

In [ ]:
book_sparse

: 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

: 

In [ ]:
imputer = SimpleImputer(strategy='mean')
book_sparse = imputer.fit_transform(book_sparse)

: 

In [ ]:
scaler = StandardScaler(with_mean=False)
book_sparse = scaler.fit_transform(book_sparse)

: 

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors (algorithm= 'brute')

: 

In [ ]:
model.fit(book_sparse)

: 

In [ ]:
book_sparse

: 

In [ ]:
book_pivot_filled = book_pivot.fillna(book_pivot.mean())


: 

In [ ]:
# Reshape and find nearest neighbors
distance, suggestion = model.kneighbors(book_pivot_filled.iloc[237, :].values.reshape(1, -1), n_neighbors=6)

: 

In [ ]:
distance

: 

In [ ]:
suggestion

: 

In [ ]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

: 

In [ ]:
books_name = book_pivot.index

: 

In [ ]:
import pickle
pickle.dump (model, open('artifacts/model.pkl', 'wb'))
pickle.dump (books_name, open('artifacts/books_name.pkl', 'wb'))
pickle.dump(fd, open('artifacts/fd.pkl', 'wb'))
pickle.dump (book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

: 

In [ ]:
import ipywidgets

: 

In [ ]:
import ipywidgets as widgets
from IPython.display import display

text = widgets.Text(value='',
                   placeholder='Type here',
                   description='String:',
                   disabled=False)

instructions = widgets.HTML('Enter book: ')
display(instructions)
display(text)

def recommend_book(book_name):
    # Your book recommendation logic goes here
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot_filled.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6)

    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        print("Recommended Books: ")
        for j in books:
            print(j)

def on_submit_handler(sender):
    recommend_book(text.value)

text.on_submit(on_submit_handler)


: 

: 